In [10]:
import numpy as np
import pandas
import os
from pycatchmod.io.json import catchment_from_json
from collections import defaultdict

In [11]:
JSON_DIR = '../hydraville/json'
DATA_DIR = '../data'
CATCHMOD_DIR = os.path.join(JSON_DIR, 'catchmod')

In [12]:
def generate_catchmod_models():

    for filename in sorted(os.listdir(CATCHMOD_DIR)):
        base, ext = os.path.splitext(filename)
        if ext != '.json':
            continue

        yield catchment_from_json(os.path.join(CATCHMOD_DIR, filename))
        
models = [c for c in generate_catchmod_models()]

C:\Anaconda3\lib\site-packages\pycatchmod\io\json.py:35: UserWarning: Small or zero linear_storage_constant is invalid. Assuming no linear store.
  legacy=data.get("legacy", False)
C:\Anaconda3\lib\site-packages\pycatchmod\io\json.py:35: UserWarning: Small or zero nonlinear_storage_constant is invalid. Assuming no non-linear store.
  legacy=data.get("legacy", False)


In [13]:


def run_catchmod(C, rainfall, temperature, dates):
    """Convenience function for running catchmod

    Parameters
    C : pycatchmod.OudinCatchment
    rainfall : numpy.array
    pet : numpy.array
    """
    import numpy as np

    C.reset()

    # number of scenarios
    N = C.subcatchments[0].soil_store.initial_upper_deficit.shape[0]
    assert(rainfall.shape[1] == N)

    # input timesteps
    M = rainfall.shape[0]
    # output timesteps
    if dates is not None:
        M2 = len(dates)
    else:
        M2 = M

    perc = np.zeros((len(C.subcatchments), N))
    outflow = np.zeros_like(perc)
    total_outflow = np.zeros(M)
    pet = np.zeros(N)
    flow = np.zeros([M2, N])
    flows = np.zeros([M2, len(C.subcatchments)])

    # TODO: add option to enable/disable extra leap days

    i = 0
    for j in range(M2):
        date = dates[j]
        if M != M2:
            if date.month == 2 and date.day == 29:
                # input data is missing leap days, use previous day
                i -= 1

        r = rainfall[i, ...].reshape(N).astype(np.float64)
        t = temperature[i, ...].reshape(N).astype(np.float64)

        C.step(date.dayofyear, r, t, pet, perc, outflow)

        flows[j, ...] = outflow[:, 0]
        flow[j, ...] = outflow.sum(axis=0).reshape(rainfall.shape[1:])

        i += 1

    return flow

In [14]:
plot = False
max_scenarios_to_process = None  # process all scenarios

# Change thise to run different sub-samples
sample_sizes = (10, 25, 50, 100, 200)
for ss in sample_sizes:
    print(f'Starting flow generation for sub-sample of size {ss}.')
    WEATHER = os.path.join(DATA_DIR, f'ukcp09_sub{ss:03d}.h5')
    FLOWS = os.path.join(DATA_DIR, f'catchmod_flows_sub{ss:03d}.h5')
    FLOWS_BY_CATCHMENT = os.path.join(DATA_DIR, f'catchmod_flows_by_catchment_sub{ss:03d}.h5')

    with pandas.HDFStore(WEATHER, mode='r') as store, \
        pandas.HDFStore(FLOWS, mode='w', complib='zlib', complevel=9) as out_store:

            i = 0
            for key in store.keys():
                if 'hly' in key:
                    continue  # Skip hourly data

                print('Predicting flow for scenario: "{}"'.format(key))
                df = store[key]

                # Massage the dates into a timeseries index.
                # NOTE: UKCP09 uses notional years starting in 3000.
                # pandas doesn't like dates this far in the future say we move them to 2000
                dates = {k: df.index.get_level_values(k) for k in ('year', 'month', 'day')}
                dates['year'] = dates['year'] - 1000
                dates = pandas.to_datetime(dates)

                rainfall = df['precip_dtotal'].values[:, np.newaxis]
                pet = df['pet_dmean'].values[:, np.newaxis]

                flows = {}
                for i, model in enumerate(models):
                    flow = run_catchmod(model, rainfall, pet, dates)
                    flow /= 1e3  # Convert to Mm3/day
                    flows[model.name] = flow[:, 0]

                flows = pandas.DataFrame(data=flows, index=dates)
                flows.name = key
                out_store[key] = flows

                if plot:
                    for ax, model in zip(axarr, models):
                        flows[model.name].plot(ax=ax)
                        flows[model.name].resample('M').mean().plot(ax=ax)
                        flows[model.name].resample('A').mean().plot(ax=ax, style='-o')

                i += 1
                if max_scenarios_to_process is not None and i >= max_scenarios_to_process:
                    break

Starting flow generation for sub-sample of size 10.
Predicting flow for scenario: "/r_0062_cntr_dly"
Predicting flow for scenario: "/r_0062_scen_dly"
Predicting flow for scenario: "/r_0136_cntr_dly"
Predicting flow for scenario: "/r_0136_scen_dly"
Predicting flow for scenario: "/r_0272_cntr_dly"
Predicting flow for scenario: "/r_0272_scen_dly"
Predicting flow for scenario: "/r_0321_cntr_dly"
Predicting flow for scenario: "/r_0321_scen_dly"
Predicting flow for scenario: "/r_0498_cntr_dly"
Predicting flow for scenario: "/r_0498_scen_dly"
Predicting flow for scenario: "/r_0586_cntr_dly"
Predicting flow for scenario: "/r_0586_scen_dly"
Predicting flow for scenario: "/r_0604_cntr_dly"
Predicting flow for scenario: "/r_0604_scen_dly"
Predicting flow for scenario: "/r_0636_cntr_dly"
Predicting flow for scenario: "/r_0636_scen_dly"
Predicting flow for scenario: "/r_0697_cntr_dly"
Predicting flow for scenario: "/r_0697_scen_dly"
Predicting flow for scenario: "/r_0799_cntr_dly"
Predicting flow f

Predicting flow for scenario: "/r_0947_scen_dly"
Predicting flow for scenario: "/r_0977_cntr_dly"
Predicting flow for scenario: "/r_0977_scen_dly"
Predicting flow for scenario: "/r_0986_cntr_dly"
Predicting flow for scenario: "/r_0986_scen_dly"
Starting flow generation for sub-sample of size 100.
Predicting flow for scenario: "/r_0008_cntr_dly"
Predicting flow for scenario: "/r_0008_scen_dly"
Predicting flow for scenario: "/r_0023_cntr_dly"
Predicting flow for scenario: "/r_0023_scen_dly"
Predicting flow for scenario: "/r_0028_cntr_dly"
Predicting flow for scenario: "/r_0028_scen_dly"
Predicting flow for scenario: "/r_0029_cntr_dly"
Predicting flow for scenario: "/r_0029_scen_dly"
Predicting flow for scenario: "/r_0030_cntr_dly"
Predicting flow for scenario: "/r_0030_scen_dly"
Predicting flow for scenario: "/r_0038_cntr_dly"
Predicting flow for scenario: "/r_0038_scen_dly"
Predicting flow for scenario: "/r_0043_cntr_dly"
Predicting flow for scenario: "/r_0043_scen_dly"
Predicting flow 

Predicting flow for scenario: "/r_0829_cntr_dly"
Predicting flow for scenario: "/r_0829_scen_dly"
Predicting flow for scenario: "/r_0852_cntr_dly"
Predicting flow for scenario: "/r_0852_scen_dly"
Predicting flow for scenario: "/r_0872_cntr_dly"
Predicting flow for scenario: "/r_0872_scen_dly"
Predicting flow for scenario: "/r_0881_cntr_dly"
Predicting flow for scenario: "/r_0881_scen_dly"
Predicting flow for scenario: "/r_0900_cntr_dly"
Predicting flow for scenario: "/r_0900_scen_dly"
Predicting flow for scenario: "/r_0903_cntr_dly"
Predicting flow for scenario: "/r_0903_scen_dly"
Predicting flow for scenario: "/r_0910_cntr_dly"
Predicting flow for scenario: "/r_0910_scen_dly"
Predicting flow for scenario: "/r_0926_cntr_dly"
Predicting flow for scenario: "/r_0926_scen_dly"
Predicting flow for scenario: "/r_0928_cntr_dly"
Predicting flow for scenario: "/r_0928_scen_dly"
Predicting flow for scenario: "/r_0948_cntr_dly"
Predicting flow for scenario: "/r_0948_scen_dly"
Predicting flow for 

Predicting flow for scenario: "/r_0293_scen_dly"
Predicting flow for scenario: "/r_0295_cntr_dly"
Predicting flow for scenario: "/r_0295_scen_dly"
Predicting flow for scenario: "/r_0298_cntr_dly"
Predicting flow for scenario: "/r_0298_scen_dly"
Predicting flow for scenario: "/r_0299_cntr_dly"
Predicting flow for scenario: "/r_0299_scen_dly"
Predicting flow for scenario: "/r_0302_cntr_dly"
Predicting flow for scenario: "/r_0302_scen_dly"
Predicting flow for scenario: "/r_0306_cntr_dly"
Predicting flow for scenario: "/r_0306_scen_dly"
Predicting flow for scenario: "/r_0313_cntr_dly"
Predicting flow for scenario: "/r_0313_scen_dly"
Predicting flow for scenario: "/r_0315_cntr_dly"
Predicting flow for scenario: "/r_0315_scen_dly"
Predicting flow for scenario: "/r_0335_cntr_dly"
Predicting flow for scenario: "/r_0335_scen_dly"
Predicting flow for scenario: "/r_0340_cntr_dly"
Predicting flow for scenario: "/r_0340_scen_dly"
Predicting flow for scenario: "/r_0343_cntr_dly"
Predicting flow for 

Predicting flow for scenario: "/r_0817_scen_dly"
Predicting flow for scenario: "/r_0822_cntr_dly"
Predicting flow for scenario: "/r_0822_scen_dly"
Predicting flow for scenario: "/r_0823_cntr_dly"
Predicting flow for scenario: "/r_0823_scen_dly"
Predicting flow for scenario: "/r_0824_cntr_dly"
Predicting flow for scenario: "/r_0824_scen_dly"
Predicting flow for scenario: "/r_0864_cntr_dly"
Predicting flow for scenario: "/r_0864_scen_dly"
Predicting flow for scenario: "/r_0867_cntr_dly"
Predicting flow for scenario: "/r_0867_scen_dly"
Predicting flow for scenario: "/r_0872_cntr_dly"
Predicting flow for scenario: "/r_0872_scen_dly"
Predicting flow for scenario: "/r_0873_cntr_dly"
Predicting flow for scenario: "/r_0873_scen_dly"
Predicting flow for scenario: "/r_0886_cntr_dly"
Predicting flow for scenario: "/r_0886_scen_dly"
Predicting flow for scenario: "/r_0902_cntr_dly"
Predicting flow for scenario: "/r_0902_scen_dly"
Predicting flow for scenario: "/r_0903_cntr_dly"
Predicting flow for 

In [15]:

for ss in sample_sizes:
    FLOWS = os.path.join(DATA_DIR, f'catchmod_flows_sub{ss:03d}.h5')
    FLOWS_BY_CATCHMENT = os.path.join(DATA_DIR, f'catchmod_flows_by_catchment_sub{ss:03d}.h5')
    
    with pandas.HDFStore(FLOWS, mode='r') as store:

            data = defaultdict(lambda: defaultdict(dict))

            i = 0
            for key in store.keys():
                if 'hly' in key:
                    continue  # Skip hourly data

                df = store[key]

                for col in df.columns:
                    if 'cntr' in key:
                        scenario = 'cntr'
                    else:
                        scenario = 'scen'    
                    data[col][scenario][key.replace("/","")] = df[col]


    with pandas.HDFStore(FLOWS_BY_CATCHMENT, mode='w', complib='zlib', complevel=9) as out_store:                

        for col in data.keys():
            for scen in data[col].keys():
                print(f'Writing combined flow for catchment-scenario: "{col}_{scen}"')
                df = pandas.concat(data[col][scen], axis=1)
                out_store[f"{col}_{scen}"] = df

Writing combined flow for catchment-scenario: "catchment1_cntr"
Writing combined flow for catchment-scenario: "catchment1_scen"
Writing combined flow for catchment-scenario: "catchment2_cntr"
Writing combined flow for catchment-scenario: "catchment2_scen"
Writing combined flow for catchment-scenario: "catchment3_cntr"
Writing combined flow for catchment-scenario: "catchment3_scen"
Writing combined flow for catchment-scenario: "catchment4_cntr"
Writing combined flow for catchment-scenario: "catchment4_scen"
Writing combined flow for catchment-scenario: "catchment5_cntr"
Writing combined flow for catchment-scenario: "catchment5_scen"
Writing combined flow for catchment-scenario: "catchment1_cntr"
Writing combined flow for catchment-scenario: "catchment1_scen"
Writing combined flow for catchment-scenario: "catchment2_cntr"
Writing combined flow for catchment-scenario: "catchment2_scen"
Writing combined flow for catchment-scenario: "catchment3_cntr"
Writing combined flow for catchment-scen